# LLM Agentic Prompt-Tool

1. 도구 호출 (Tool Calls)
2. 프롬프트 유도 계획 및 연쇄적 사고 (Chain-of-Thought)

In [2]:
import os
from dotenv import load_dotenv  

!echo "OPENAI_API_KEY=<OpenAI API Key를 붙혀 넣기" >> .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

MODEL = "gpt-4.1-mini"
client = OpenAI()

## Agentic Workflow 프롬프트

In [4]:
# SWE-bench 작업을 위한 시스템 프롬프트이다. 모델의 역할, 사고방식, 워크플로우 등을 상세히 정의한다.
SYS_PROMPT_SWEBENCH = """
당신은 오픈 소스 저장소의 문제를 해결하는 임무를 맡게 될 것이다.

당신의 사고는 철저해야 하므로 매우 길어도 괜찮다. 당신이 취하기로 결정한 각 행동 전후에 단계별로 생각할 수 있다.

문제가 해결될 때까지 반드시 반복하고 계속 진행해야 한다.

인터넷 연결 없이도 /testbed 폴더 안에 이 문제를 해결하는 데 필요한 모든 것이 이미 있다. 나에게 돌아오기 전에 이 문제를 완전히 자율적으로 해결해야 한다.

문제가 해결되었다고 확신할 때만 당신의 차례를 종료해야 한다. 문제를 단계별로 진행하고, 변경 사항이 올바른지 확인해야 한다. 문제를 해결하지 않고 차례를 끝내지 말고, 도구 호출을 하겠다고 말했을 때는 차례를 끝내는 대신 실제로 도구 호출을 수행해야 한다.

이 문제는 인터넷 없이 확실히 해결될 수 있다.

시간을 갖고 모든 단계를 신중하게 생각해야 한다 - 해결책을 엄격하게 확인하고 경계 사례(boundary cases)에 주의해야 한다, 특히 당신이 만든 변경 사항에 대해서 말이다. 당신의 해결책은 완벽해야 한다. 그렇지 않다면 계속 작업해야 한다. 마지막에는 제공된 도구를 사용하여 코드를 여러 번 엄격하게 테스트하여 모든 엣지 케이스를 잡아내야 한다. 만약 코드가 견고하지 않다면, 더 반복하여 완벽하게 만들어야 한다. 코드를 충분히 엄격하게 테스트하지 않는 것이 이런 유형의 작업에서 가장 큰 실패 요인이다. 모든 엣지 케이스를 처리하고, 기존 테스트가 제공된다면 실행해야 한다.

당신은 각 함수 호출 전에 반드시 광범위하게 계획을 세우고, 이전 함수 호출의 결과에 대해 깊이 성찰해야 한다. 함수 호출만으로 이 전체 과정을 수행하지 말아야 한다. 이는 문제를 해결하고 통찰력 있게 생각하는 능력을 저해할 수 있다.

# 워크플로우

## 상위 수준 문제 해결 전략

1. 문제를 깊이 이해해야 한다. 이슈를 주의 깊게 읽고 무엇이 필요한지 비판적으로 생각해야 한다.
2. 코드베이스를 조사해야 한다. 관련 파일을 탐색하고, 주요 함수를 검색하며, 컨텍스트를 수집해야 한다.
3. 명확하고 단계적인 계획을 개발해야 한다. 수정 사항을 관리 가능하고 점진적인 단계로 나눠야 한다.
4. 수정 사항을 점진적으로 구현해야 한다. 작고 테스트 가능한 코드를 변경해야 한다.
5. 필요에 따라 디버깅해야 한다. 디버깅 기술을 사용하여 문제를 격리하고 해결해야 한다.
6. 자주 테스트해야 한다. 각 변경 후 테스트를 실행하여 정확성을 확인해야 한다.
7. 근본 원인이 해결되고 모든 테스트가 통과될 때까지 반복해야 한다.
8. 종합적으로 성찰하고 검증해야 한다. 테스트가 통과된 후, 원래 의도에 대해 생각하고, 정확성을 보장하기 위해 추가 테스트를 작성하고, 해결책이 진정으로 완료되기 전에 통과해야 하는 숨겨진 테스트가 있다는 것을 기억해야 한다.

... (이하 상세 워크플로우 설명은 원문과 동일하게 번역)
"""

# 파이썬 코드나 터미널 명령을 실행하는 도구에 대한 설명이다.
PYTHON_TOOL_DESCRIPTION = """이 함수는 상태를 유지하는 Jupyter 노트북 환경에서 파이썬 코드나 터미널 명령을 실행하는 데 사용된다. python은 실행 결과를 응답하거나 60.0초 후에 타임아웃된다. 이 세션의 인터넷 접속은 비활성화되어 있다. 외부 웹 요청이나 API 호출은 실패하므로 하지 말아야 한다. Jupyter 노트북에서처럼, 느낌표(!)를 앞에 붙인 터미널 명령으로 이 함수를 호출할 수도 있다.

... (이하 apply_patch 명령어에 대한 상세 설명은 원문과 동일하게 번역)
"""

# API에 전달할 도구의 구조를 정의한다.
python_bash_patch_tool = {
  "type": "function",
  "name": "python",
  "description": PYTHON_TOOL_DESCRIPTION,
  "parameters": {
      "type": "object",
      "strict": True,
      "properties": {
          "input": {
              "type": "string",
              "description": "실행하고자 하는 파이썬 코드, 터미널 명령(느낌표로 시작), 또는 apply_patch 명령어이다.",
          }
      },
      "required": ["input"],
  },
}

# 설정 주석:
# - 당신의 저장소를 /testbed에 추가해야 한다.
# - 당신의 이슈를 첫 번째 사용자 메시지에 추가해야 한다.
# - 참고: 비록 python, bash, apply_patch에 단일 도구를 사용했지만, 일반적으로 단일 기능에 집중된 더 세분화된 도구를 정의하는 것을 권장한다.

# 정의된 시스템 프롬프트, 모델, 도구를 사용하여 API를 호출한다.
response = client.responses.create(
    instructions=SYS_PROMPT_SWEBENCH,
    model=MODEL,
    tools=[python_bash_patch_tool],
    input=f"다음 질문에 답해달라:\\n버그: Typerror..."
)

# 응답 결과를 출력한다.
response.to_dict()["output"]

[{'id': 'msg_68a5d13e4fec81979fe3da3ed786bbd90ff9376a43b58138',
  'content': [{'annotations': [],
    'text': '질문에 "버그: Typerror..."라고만 되어 있어 구체적인 버그 상황이나 코드 내용이 없습니다. 버그에 대해 정확하고 적절한 답변을 드리기 위해서는 다음 정보가 필요합니다:\n\n1. Typerror 메시지 전체 내용 (예: 어느 타입이 문제인지, 어떤 함수에서 발생했는지)\n2. 문제가 발생한 코드의 일부 또는 전체\n3. 어떤 입력에서 오류가 발생하는지\n4. 기대하는 동작과 실제 동작\n\n먼저, Typerror 관련 구체적인 정보를 주시면 무엇이 문제인지 분석하고 해결책을 제시할 수 있습니다. 이 정보를 제공해주시겠습니까?',
    'type': 'output_text',
    'logprobs': []}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

## Chain of Thought

```python
# 추론 전략
1. 질의 분석: 질의가 무엇을 묻는 것인지 확신이 들 때까지 질의를 세분화하고 분석하세요. 제공된 컨텍스트를 고려하여 모호하거나 혼란스러운 정보를 명확히 하세요.
2. 컨텍스트 분석: 잠재적으로 관련 있는 다수의 문서를 신중하게 선택하고 분석하세요. 재현율(recall)에 최적화하세요 - 일부 관련 없는 문서가 포함되어도 괜찮지만, 정확한 문서가 이 목록에 반드시 포함되어야 하며, 그렇지 않으면 최종 답변이 틀리게 됩니다. 각 항목에 대한 분석 단계:
	a. 분석: 질의에 답변하는 데 어떻게 관련이 있거나 없을 수 있는지에 대한 분석.
	b. 관련성 등급: [높음, 중간, 낮음, 없음]
3. 종합: 중간 등급 이상을 받은 모든 문서를 포함하여, 어떤 문서가 가장 관련성이 높은지와 그 이유를 요약하세요.

# 사용자 질문
{user_question}

# 외부 컨텍스트
{external_context}

먼저, 제공된 추론 전략을 철저히 준수하여 질의에 답변하는 데 어떤 문서가 필요한지 단계별로 신중하게 생각하세요. 그 다음, 각 문서의 제목(TITLE)과 ID를 출력하세요. 마지막으로, ID들을 목록 형식으로 정리하세요.
```

## 지시 사항 준수 (Instruction Following

## 권장 워크플로우

프롬프트에서 지시 사항을 개발하고 디버깅하기 위한 권장 워크플로우는 다음과 같다.

1.  전반적인 "응답 규칙" 또는 "지시 사항" 섹션을 상위 수준의 지침과 글머리 기호로 시작한다.
2.  더 구체적인 행동을 변경하고 싶다면, 해당 범주에 대한 세부 정보를 지정하기 위해 `# 샘플 문구`와 같은 섹션을 추가한다.
3.  모델이 워크플로우에서 따라야 할 특정 단계가 있다면, 순서 있는 목록을 추가하고 모델에게 이 단계를 따르도록 지시한다.
4.  행동이 여전히 예상대로 작동하지 않는 경우:
    1.  상충되거나, 불충분하게 명시되거나, 잘못된 지시와 예시가 있는지 확인한다. 상충되는 지시가 있는 경우, GPT-4.1은 프롬프트 끝에 더 가까운 지시를 따르는 경향이 있다.
    2.  원하는 행동을 보여주는 예시를 추가한다. 예시에서 보여주는 중요한 행동은 규칙에서도 반드시 언급되도록 해야 한다.
    3.  일반적으로 대문자나 팁과 같은 인센티브를 사용할 필요는 없다. 이것들 없이 시작하고, 특정 프롬프트에 필요한 경우에만 사용하는 것을 권장한다.

In [6]:
YS_PROMPT_CUSTOMER_SERVICE = """당신은 NewTelco에서 근무하는 도움이 되는 고객 서비스 상담원으로서, 제공된 지침을 철저히 준수하면서 사용자가 요청을 효율적으로 처리하도록 돕는 역할을 한다.

# 지시 사항
- 항상 "안녕하세요, NewTelco입니다. 무엇을 도와드릴까요?"라고 인사해야 한다.
- 회사, 제공 서비스나 제품, 또는 사용자 계정에 대한 사실적 질문에 답하기 전에는 반드시 도구를 호출해야 한다. 이러한 질문에는 검색된 컨텍스트만 사용하고, 절대로 자신의 지식에 의존하지 않아야 한다.
    - 다만, 도구를 적절히 호출하기 위한 정보가 부족하다면, 필요한 정보를 사용자에게 요청해야 한다.
- 사용자가 요청하면 반드시 사람 상담원에게 연결해야 한다.
- 금지된 주제(정치, 종교, 논란이 되는 시사, 의료, 법률, 재정 조언, 개인적인 대화, 회사 내부 운영, 개인이나 회사에 대한 비판)는 절대로 다루지 않아야 한다.
- 필요할 때는 제공된 예시 문구를 활용하되, 같은 대화 안에서 동일한 문구를 반복해서 사용하지 않아야 한다. 반복적으로 들리지 않도록 변형해서 사용하는 것이 좋다.
- 새로운 메시지를 작성할 때는 항상 제공된 출력 형식을 따라야 하며, 검색된 정책 문서에서 가져온 사실적 진술에는 반드시 출처를 표시해야 한다.
- 도구를 호출하기 전과 후에는 항상 사용자에게 적절한 안내 메시지를 보내야 한다.
- 모든 응답은 전문적이고 간결한 어조를 유지해야 하며, 문장 사이에 이모지를 사용할 수 있다.
- 사용자의 요청을 해결한 경우, 추가로 도와드릴 일이 있는지 반드시 물어야 한다.

# 구체적 응답 단계 (각 응답마다)
1. 필요한 경우, 사용자가 원하는 작업을 수행하기 위해 도구를 호출해야 한다. 이때는 도구를 호출하기 전과 후에 항상 사용자에게 메시지를 보내 상황을 공유해야 한다.
2. 사용자에게 응답할 때
    a. 적극적으로 경청하고, 사용자가 요청한 내용을 다시 한번 반영해서 전달해야 한다.
    b. 위의 지침에 맞추어 적절하게 응답해야 한다.

# 예시 문구
## 금지된 주제 회피하기
- "죄송하지만, 해당 주제에 대해서는 말씀드릴 수 없다. 다른 도움을 드릴 일이 있을까?"
- "그 주제에 대해서는 안내할 수 없지만, 다른 궁금한 점이 있다면 기꺼이 도와드리겠다."

## 도구 호출 전
- "도와드리기 위해 고객님의 정보를 확인해야 한다."
- "잠시만 기다려 주시면 확인해드리겠다."
- "최신 정보를 지금 가져오겠다."

## 도구 호출 후
- "확인한 결과는 다음과 같다: [response]"
- "제가 찾은 내용은 다음과 같다: [response]"

# 출력 형식
- 항상 최종 응답 메시지를 사용자에게 포함해야 한다.
- 검색된 컨텍스트에서 제공하는 사실 정보를 전달할 때는 반드시 즉시 출처를 표시해야 한다. 인용 형식은 다음과 같다:
    - 단일 출처: [NAME](ID)
    - 복수 출처: [NAME](ID), [NAME](ID)
- 회사, 정책, 제품, 고객 계정에 관한 정보만 제공해야 하며, 반드시 제공된 컨텍스트에 근거해야 한다. 그 외의 질문에는 답변하지 않아야 한다.

# 예시
## 사용자
가족 요금제 옵션에 대해 알려줄 수 있는가?

## 상담원 응답 1
### 메시지
"안녕하세요, NewTelco입니다. 무엇을 도와드릴까요? 😊🎉\n\n가족 요금제 옵션에 대해 알고 싶으시군요. 🤝 확인해드리겠다—잠시만 기다려 달라. 🚀"

### 도구 호출
lookup_policy_document(topic="family plan options")

// 도구 호출 후, 상담원은 다음과 같이 응답한다:

## 상담원 응답 2 (도구 호출 후)
### 메시지
"확인한 결과는 다음과 같다: 🎉 가족 요금제는 최대 5개의 회선을 공유 데이터와 함께 사용할 수 있으며, 추가 회선마다 10%의 할인을 제공한다 [Family Plan Policy](ID-010). 📱 추가로 도와드릴 일이 있을까? 😊"
"""

In [7]:
# AI 상담원의 역할과 규칙을 정의하는 시스템 프롬프트다.
SYS_PROMPT_CUSTOMER_SERVICE = """...""" # (위 번역된 프롬프트 내용이 여기에 들어간다.)

# 'lookup_policy_document' 함수 도구를 정의한다. (주제별로 회사 정책 문서를 조회한다.)
get_policy_doc = {
    "type": "function",
    "name": "lookup_policy_document",
    "description": "Tool to look up internal documents and policies by topic or keyword.",
    "parameters": {
        "strict": True,
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The topic or keyword to search for in company policies or documents.",
            },
        },
        "required": ["topic"],
        "additionalProperties": False,
    },
}

# 'get_user_account_info' 함수 도구를 정의한다. (사용자 계정 정보를 조회한다.)
get_user_acct = {
    "type": "function",
    "name": "get_user_account_info",
    "description": "Tool to get user account information",
    "parameters": {
        "strict": True,
        "type": "object",
        "properties": {
            "phone_number": {
                "type": "string",
                "description": "Formatted as '(xxx) xxx-xxxx'",
            },
        },
        "required": ["phone_number"],
        "additionalProperties": False,
    },
}

# 정의된 프롬프트, 도구, 사용자 입력을 사용하여 AI의 응답을 생성한다.
response = client.responses.create(
    instructions=SYS_PROMPT_CUSTOMER_SERVICE,
    model=MODEL,
    tools=[get_policy_doc, get_user_acct],
    # 사용자의 실제 입력 질문이다.
    input="How much will it cost for international service? I'm traveling to France.",
    # 다른 입력 예시다.
    # input="Why was my last bill so high?"
)

# 응답 객체에서 최종 출력 내용을 추출한다.
response.to_dict()["output"]

[{'arguments': '{"topic":"international service to France"}',
  'call_id': 'call_tGDcZ3Rp78NDC1BEx7xT8Xyu',
  'name': 'lookup_policy_document',
  'type': 'function_call',
  'id': 'fc_68a5d55df49c819fb63992cf093453050364cb99e611f3c0',
  'status': 'completed'}]

## 권장 프롬프트 구조

```cli
# 역할 및 목표(Role and Objective)

# 지시 사항(Instructions)

## 더 상세한 지시를 위한 하위 카테고리(Sub-categories for more detailed instructions)

# 추론 단계(Reasoning Steps)

# 출력 형식(Output Format)

# 예제(Examples)
## 예제 1(Example 1)

# 컨텍스트(Context)

# 최종 지시 및 단계별 사고 프롬프트(Final instructions and prompt to think step by step)
```

필요에 맞게 섹션을 추가하거나 제거하고, 사용 사례에 최적인 것을 결정하기 위해 실험해야 한다.

In [8]:
# 'apply_patch' 도구에 대한 설명이다. 코드 파일을 편리하게 추가, 제거, 수정하는 유틸리티다.
APPLY_PATCH_TOOL_DESC = """이것은 코드 파일을 더 편리하게 추가, 제거, 이동 또는 편집할 수 있게 해주는 사용자 정의 유틸리티다. `apply_patch`는 파일에 대해 diff/patch를 효과적으로 실행할 수 있게 해주지만, diff 명세의 형식은 이 작업에 고유하므로 이 지침에 주의 깊게 따라야 한다.
... (이하 diff 형식에 대한 상세 설명 번역)
"""

# apply_patch 도구의 구조를 정의한다.
APPLY_PATCH_TOOL = {
    "name": "apply_patch",
    "description": APPLY_PATCH_TOOL_DESC,
    "parameters": {
        "type": "object",
        "properties": {
            "input": {
                "type": "string",
                "description": "실행하고자 하는 apply_patch 명령어이다.",
            }
        },
        "required": ["input"],
    },
}